# Proportional Election Simulations

**Prepare Workspace**

In [114]:
# Import libraries and set appearance parameters
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import datasets
df = pd.read_csv('Election Results.csv', thousands=',')
df_sum = pd.read_csv('Summary.csv', thousands=',')

# Declare global variables
pt = list(df['Province/Territory'].unique())

# Hide extra warnings
import warnings
warnings.filterwarnings(action='ignore')

**1. Dual Member Proportional**

In [53]:
# Build new dataframe
cols = ['LPC', 'CPC', 'BQ', 'NDP', 'GP', 'PP', 'Other', 'Turnout', 'Area']
df_dmp = pd.DataFrame(columns=cols)

# Build new ridings
for i in pt:
    prov = df[df['Province/Territory'] == i][cols]
    dual = df[df['Province/Territory'] == i][df['Area'] <= 1500][cols]
    
    if len(dual) % 2 == 1:
        dual = dual.sort_values('Area').drop(dual.sort_values('Area').tail(1).index)
    
    single = prov[~prov.isin(dual)].dropna(how='all')
    combined = dual.groupby(np.arange(len(dual))//2).sum()
    combined['System'], single['System'] = 'Combined', 'Single'
    combined['PT'], single['PT'] = i, i
    
    df_dmp = df_dmp.append([combined, single], sort=False).fillna(0)

# Calculate percentage of votes
parties = ['LPC', 'CPC', 'BQ', 'NDP', 'GP', 'PP', 'Other']
for i in parties:
    df_dmp['%' + i] = round(df_dmp[i] / df_dmp['Turnout'] * 100, 1)

# Calculate first seat winners
df_dmp['Highest'] = df_dmp[parties].max(axis=1)
df_dmp['Highest Party'], df_dmp['First Seat'] = np.NaN, np.NaN

for i in parties:
    df_dmp['Highest Party'] = np.where(df_dmp['Highest'] == df_dmp[i], i, df_dmp['Highest Party'])
df_dmp['First Seat'] = df_dmp['Highest Party']

# Calculate second seat winners
for i in parties:
    df_dmp[i + ' Round 2'] = np.where(df_dmp['First Seat'] == i, df_dmp[i]/2, df_dmp[i])

round2 = ['LPC Round 2', 'CPC Round 2', 'BQ Round 2', 'NDP Round 2', 'GP Round 2', 'PP Round 2', 'Other Round 2']
df_dmp['Highest 2'] = df_dmp[round2].max(axis=1)
df_dmp['Highest Party 2'], df_dmp['Second Seat'] = np.NaN, np.NaN
for i in round2:
    df_dmp['Highest Party 2'] = np.where(df_dmp['Highest 2'] == df_dmp[i], i, df_dmp['Highest Party 2'])
    
df_dmp['Second Seat'] = np.where(df_dmp['System'] == 'Combined', df_dmp['Highest Party 2'], df_dmp['Second Seat'])
df_dmp['Second Seat'] = df_dmp['Second Seat'].str[:-8]

# Count total seats
seat1 = pd.DataFrame(df_dmp['First Seat'].value_counts())
seat2 = pd.DataFrame(df_dmp['Second Seat'].value_counts())
total = seat1.join(seat2).fillna(0)
total['Total'] = total['First Seat'] + total['Second Seat']

**2. MMP**

In [134]:
# Build new dataframe
cols = ['LPC', 'CPC', 'BQ', 'NDP', 'GP', 'PP', 'Other', 'Turnout', 'Area']
df_dmp = pd.DataFrame(columns=cols)
df_mmp = pd.DataFrame(columns=cols)

# Build new ridings
for i in pt:
    prov = df[df['Province/Territory'] == i][cols]
    dual = df[df['Province/Territory'] == i][df['Area'] <= 1500][cols]
    
    if len(dual) % 2 == 1:
        dual = dual.sort_values('Area').drop(dual.sort_values('Area').tail(1).index)
    
    single = prov[~prov.isin(dual)].dropna(how='all')
    combined = dual.groupby(np.arange(len(dual))//2).sum()
    combined['System'], single['System'] = 'Combined', 'Single'
    combined['PT'], single['PT'] = i, i
    
    df_dmp = df_dmp.append([combined, single], sort=False).fillna(0)

df_dmp = df_dmp.reset_index(drop=True)
cols = list(df_dmp.columns)
for i in pt:
    prov = df_dmp[df_dmp['PT'] == i][cols]
    quad = df_dmp[df_dmp['PT'] == i][df_dmp['Area'] <= 1500][df_dmp['System'] == 'Combined'][cols]
    
    if len(quad) % 2 == 1:
        quad = quad.sort_values('Area').drop(quad.sort_values('Area').tail(1).index)
    
    nonquad = prov[~prov.isin(quad)].dropna(how='all')
    double_combined = quad.groupby(np.arange(len(quad))//2).sum()
    double_combined['System'], double_combined['PT'] = 'Double Combined', i
    
    df_mmp = df_mmp.append([double_combined, nonquad], sort=False).fillna(0)

# Calculate percentage of votes
parties = ['LPC', 'CPC', 'BQ', 'NDP', 'GP', 'PP', 'Other']
for i in parties:
    df_mmp['%' + i] = round(df_mmp[i] / df_mmp['Turnout'] * 100, 1)
    


In [135]:
df_mmp

,LPC,CPC,BQ,NDP,GP,PP,Other,Turnout,Area,System,PT,%LPC,%CPC,%BQ,%NDP,%GP,%PP,%Other
0,59615.0,107922.0,0.0,34732.0,6204.0,3615.0,1985.0,214073.0,191.0,Double Combined,AB,27.8,50.4,0.0,16.2,2.9,1.7,0.9
1,45196.0,131674.0,0.0,25366.0,9970.0,4103.0,1409.0,217718.0,224.0,Double Combined,AB,20.8,60.5,0.0,11.7,4.6,1.9,0.6
2,30723.0,152967.0,0.0,42541.0,6979.0,4659.0,1301.0,239170.0,303.0,Double Combined,AB,12.8,64.0,0.0,17.8,2.9,1.9,0.5
3,50607.0,147658.0,0.0,31303.0,5639.0,3989.0,1375.0,240571.0,422.0,Double Combined,AB,21.0,61.4,0.0,13.0,2.3,1.7,0.6
8,19364.0,86630.0,0.0,15563.0,3396.0,2735.0,322.0,128010.0,344.0,Combined,AB,15.1,67.7,0.0,12.2,2.7,2.1,0.3
9,8216.0,54580.0,0.0,7960.0,3230.0,2609.0,0.0,76595.0,12282.0,Single,AB,10.7,71.3,0.0,10.4,4.2,3.4,0.0
10,2485.0,52497.0,0.0,3089.0,1750.0,1583.0,0.0,61404.0,53072.0,Single,AB,4.0,85.5,0.0,5.0,2.8,2.6,0.0
11,3104.0,45691.0,0.0,3010.0,805.0,1302.0,445.0,54357.0,24427.0,Single,AB,5.7,84.1,0.0,5.5,1.5,2.4,0.8
12,10300.0,61709.0,0.0,9623.0,1609.0,1641.0,211.0,85093.0,4947.0,Single,AB,12.1,72.5,0.0,11.3,1.9,1.9,0.2
13,3765.0,52907.0,0.0,3641.0,2309.0,1668.0,0.0,64290.0,20877.0,Single,AB,5.9,82.3,0.0,5.7,3.6,2.6,0.0


In [131]:
#(len(df_mmp[df_mmp['System'] == 'Double Combined']) * 4) + (len(df_mmp[df_mmp['System'] == 'Combined']) * 2) + len(df_mmp[df_mmp['System'] == 'Single'])

338

**3. URP**

In [21]:
df

,Province/Territory,Riding,LPC,CPC,BQ,NDP,GP,PP,Other,Turnout,Successful Voters,% Successful,Unsuccessful Voters,% Unsuccessful,Elected,% LPC,Population,Electors,Area
0,AB,Banff-Airdrie,8216,54580,-,7960,3230,"2,609",-,76595,54580,71.30%,22015,28.70%,CPC,10.70%,105442,87666,12282
1,AB,Battle River-Crowfoot,2485,52497,-,3089,1750,"1,583",-,61404,52497,85.50%,8907,14.50%,CPC,4.00%,107140,79873,53072
2,AB,Bow River,3104,45691,-,3010,805,"1,302",445,54357,45691,84.10%,8666,15.90%,CPC,5.70%,103871,73598,24427
3,AB,Calgary Centre,17199,36360,-,6290,2723,867,495,63934,36360,56.90%,27574,43.10%,CPC,26.90%,108931,83497,49
4,AB,Calgary Confederation,14462,35648,-,7113,5488,"1,117",517,64345,35648,55.40%,28697,44.60%,CPC,22.50%,111785,85997,54
5,AB,Calgary Forest Lawn,8537,23585,-,4133,1282,"1,069",825,39431,23585,59.80%,15846,40.20%,CPC,21.70%,108251,73857,53
6,AB,Calgary Heritage,7671,39098,-,5038,1917,"1,076",402,55202,39098,70.80%,16104,29.20%,CPC,13.90%,108320,79349,70
7,AB,Calgary Midnapore,7320,49700,-,6193,1942,"1,562",-,66717,49700,74.50%,17017,25.50%,CPC,11.00%,111227,85280,87
8,AB,Calgary Nose Hill,8606,37950,-,5107,1478,"1,073",67,54281,37950,69.90%,16331,30.10%,CPC,15.90%,109286,80629,57
9,AB,Calgary Rocky Ridge,12694,47477,-,5869,1927,"1,026",265,69258,47477,68.60%,21781,31.40%,CPC,18.30%,108901,84093,90


In [19]:
df_sum

,Party,Party Votes,Total Votes,Percent of Total Votes,Elected,Seats if Proportional
0,LPC,5911617.0,17873038.0,33.1%,157.0,112.0
1,CPC,6153341.0,17873038.0,34.4%,121.0,116.0
2,BQ,1377205.0,17873038.0,7.7%,32.0,26.0
3,NDP,2843661.0,17873038.0,15.9%,24.0,54.0
4,GP,1157758.0,17873038.0,6.5%,3.0,22.0
5,PP,292802.0,17873038.0,1.6%,0.0,6.0
6,Other,136654.0,17873038.0,0.8%,1.0,3.0
7,NaN,NaN,NaN,NaN,NaN,NaN
